## Objetivo del desafío
La empresa ADL se dedica a la venta de productos a nivel regional en los Estados Unidos,
con equipos de ventas que operan en diferentes estados. La compañía ha establecido un
sistema de comisiones para incentivar a sus equipos de ventas a preferir la venta de
productos de alto margen antes que los de bajo margen de utilidad.

Como Analista de Datos, le solicitan analizar información sobre las ventas realizadas por
los equipos de ventas de ADL, incluyendo detalles sobre los productos vendidos, los
clientes, las fechas de los pedidos y los plazos de entrega.

Para resolver este desafío deberás cargar datos desde Hojas de Excel del archivo
US_Regional_Sales_Data.xlsx. En base a esto, debe generar una visualización de datos
como la que se muestra en las hojas Reporte1 y Reporte2 del archivo reporte.xlsx.



## 1. Carga los datos de cada pestaña 
del archivo US_Regional_Sales_Data.xlsx en un DataFrame independiente. (hint: Puedes obtener los nombres de las hojas usando pd.ExcelFile(archivo).sheet_names)


In [3]:
import pandas as pd

#Cargar archivo
archivo_desafio = "US_Regional_Sales_Data.xlsx"

excel_file = pd.ExcelFile(archivo_desafio)

In [4]:
#Explorar archivo
print("Hojas disponibles:", excel_file.sheet_names)


Hojas disponibles: ['Sales Orders Sheet', 'Customers Sheet', 'Store Locations Sheet', 'Products Sheet', 'Sales Team Sheet']


In [5]:
# Cargar cada hoja en un DataFrame independiente
df_sales_orders = pd.read_excel(archivo_desafio, sheet_name="Sales Orders Sheet")
df_customers = pd.read_excel(archivo_desafio, sheet_name="Customers Sheet")
df_store_locations = pd.read_excel(archivo_desafio, sheet_name="Store Locations Sheet")
df_products = pd.read_excel(archivo_desafio, sheet_name="Products Sheet")
df_sales_team = pd.read_excel(archivo_desafio, sheet_name="Sales Team Sheet")



## 2. Cruza todos los DataFrames 
usando validación many_to_one, y guarda el resultado en una nueva variable llamada df_base.


In [7]:
# "Sales orders es la hoja principal que posee informacion vinculada a las otras pestañas
# Identificar la columna  en comun con todos los dataframes para unirlas
common_columns = set(df_sales_orders.columns) & set(df_customers.columns) #Interseccion de nombres de columnas en un set
print("La columna comun es:", common_columns)
common_columns = set(df_sales_orders.columns) & set(df_store_locations.columns) #Interseccion de nombres de columnas en un set
print("La columna comun es:", common_columns)
common_columns = set(df_sales_orders.columns) & set(df_products.columns) #Interseccion de nombres de columnas en un set
print("La columna comun es:", common_columns)
common_columns = set(df_sales_orders.columns) & set(df_sales_team.columns) #Interseccion de nombres de columnas en un set
print("La columna comun es:", common_columns)


La columna comun es: {'_CustomerID'}
La columna comun es: {'_StoreID'}
La columna comun es: {'_ProductID'}
La columna comun es: {'_SalesTeamID'}


In [8]:
# Unir tablas una por una usando los IDs correspondientes
df_base = (
    df_sales_orders # Pestaña principal
    .merge(df_customers, left_on="_CustomerID", right_on="_CustomerID", validate="many_to_one")
    .merge(df_store_locations, left_on="_StoreID", right_on="_StoreID", validate="many_to_one")
    .merge(df_products, left_on="_ProductID", right_on="_ProductID", validate="many_to_one")
    .merge(df_sales_team, left_on="_SalesTeamID", right_on="_SalesTeamID", validate="many_to_one")
)
df_base.head(5)

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,...,AreaCode,Population,Household Income,Median Income,Land Area,Water Area,Time Zone,Product Name,Sales Team,Region
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,...,631,213776,68789,80327,135481314,160302131,America/New York,Dining Furniture,Joshua Bennett,Northeast
1,SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,...,913,186515,74830,72463,194562941,1309517,America/Chicago,Wreaths,Paul Holmes,Midwest
2,SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,...,734,117070,47179,55990,72727009,2254528,America/Detroit,Stemware,Samuel Fowler,Midwest
3,SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,...,203,130322,49771,37192,48407373,3739487,America/New York,Accessories,Carlos Miller,West
4,SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,...,203,108802,40213,40467,73880017,1086045,America/New York,Candles,Joe Price,Northeast


## 3. Agrega las siguientes columnas a df_ base:


In [10]:
#Asegurar que las fechas esten en formato datetime
col_fechas = ["OrderDate", "ProcuredDate", "ShipDate", "DeliveryDate"]
df_base[col_fechas] = df_base[col_fechas].apply(pd.to_datetime) # Tipo datetime queda en  formato Año-mes-dia
print(df_base[col_fechas])


      OrderDate ProcuredDate   ShipDate DeliveryDate
0    2018-05-31   2017-12-31 2018-06-14   2018-06-19
1    2018-05-31   2017-12-31 2018-06-22   2018-07-02
2    2018-05-31   2017-12-31 2018-06-21   2018-07-01
3    2018-05-31   2017-12-31 2018-06-02   2018-06-07
4    2018-05-31   2018-04-10 2018-06-16   2018-06-26
...         ...          ...        ...          ...
7986 2020-12-30   2020-09-26 2021-01-07   2021-01-14
7987 2020-12-30   2020-09-26 2021-01-02   2021-01-04
7988 2020-12-30   2020-09-26 2021-01-23   2021-01-26
7989 2020-12-30   2020-09-26 2021-01-20   2021-01-25
7990 2020-12-30   2020-09-26 2021-01-13   2021-01-19

[7991 rows x 4 columns]


In [11]:
# Calcular diferencias de días (objetos timedelta)
df_base["ProcurementDays"] = df_base["OrderDate"] - df_base["ProcuredDate"]
df_base["ShippingDays"] = df_base["ShipDate"] - df_base["OrderDate"]
df_base["DeliveryDays"] = df_base["DeliveryDate"] - df_base["ShipDate"]
df_base["CustomerDays"] = df_base["ShippingDays"] + df_base["DeliveryDays"]

delta_fechas = ["ProcurementDays", "ShippingDays", "DeliveryDays", "CustomerDays"]
print(df_base[delta_fechas])

     ProcurementDays ShippingDays DeliveryDays CustomerDays
0           151 days      14 days       5 days      19 days
1           151 days      22 days      10 days      32 days
2           151 days      21 days      10 days      31 days
3           151 days       2 days       5 days       7 days
4            51 days      16 days      10 days      26 days
...              ...          ...          ...          ...
7986         95 days       8 days       7 days      15 days
7987         95 days       3 days       2 days       5 days
7988         95 days      24 days       3 days      27 days
7989         95 days      21 days       5 days      26 days
7990         95 days      14 days       6 days      20 days

[7991 rows x 4 columns]


## 4. Agrega una nueva columna discreta al df_base,
con el nombre CustomerDaysInterval que clasifique los valores de la columna CustomerDays en los siguientes intervalos:
○ 0 to 15 days
○ 15 to 30 days
○ 30 to 45 days
○ 45 to 60 days
○ 60 to 75 days
○ 75 to 90 days


In [13]:
# Convertir días a número enteros los días de customer days (tiempo de envio total)
df_base["CustomerDaysNum"] = df_base["CustomerDays"].dt.days
print(df_base["CustomerDaysNum"])

0       19
1       32
2       31
3        7
4       26
        ..
7986    15
7987     5
7988    27
7989    26
7990    20
Name: CustomerDaysNum, Length: 7991, dtype: int64


In [14]:
#Definir intervalos y etiquetas para clasificar los customer days  
intervalos = [0, 15, 30, 45, 60, 75, 90]
etiquetas = ["0 to 15 days", "15 to 30 days", "30 to 45 days", 
          "45 to 60 days", "60 to 75 days", "75 to 90 days"]

df_base["CustomerDaysInterval"] = pd.cut(
    df_base["CustomerDaysNum"],
    bins=intervalos,
    labels=etiquetas,
    right=True, #No se incluye el valor superiro del intervalo
    include_lowest=False 
)

## 5. Utiliza una tabla pivote para mostrar el conteo de órdenes
agrupando por Sales Team en las filas y CustomerDaysInterval en las columnas. Este reporte debe
cuadrar con los datos de la pestaña Reporte1 del archivo reportes.xlsx

In [16]:
# Calculo manual para corroborar valores
menos_15_dias = df_base[df_base["CustomerDaysNum"] <= 15]
print(menos_15_dias['Sales Team'].value_counts().sort_index())

Sales Team
Adam Hernandez          81
Anthony Berry           76
Anthony Torres          81
Carl Nguyen             96
Carlos Miller           65
Chris Armstrong         92
Donald Reynolds         97
Douglas Tucker          87
Frank Brown             85
George Lewis           104
Jerry Green             85
Joe Price               87
Jonathan Hawkins        81
Joshua Bennett          85
Joshua Little           83
Joshua Ryan             83
Keith Griffin           94
Nicholas Cunningham     88
Patrick Graham          92
Paul Holmes             61
Roger Alexander         82
Roy Rice                75
Samuel Fowler           90
Shawn Cook              88
Shawn Torres            77
Shawn Wallace           95
Stephen Payne           97
Todd Roberts           113
Name: count, dtype: int64


In [17]:
# Calculo manual para corroborar clasificacion
menos_15_dias = df_base[df_base["CustomerDaysInterval"] == "0 to 15 days"]
print(menos_15_dias['Sales Team'].value_counts().sort_index())
#Esto revelo un error en la clasificacion en pd.cut (right=true)

Sales Team
Adam Hernandez          81
Anthony Berry           76
Anthony Torres          81
Carl Nguyen             96
Carlos Miller           65
Chris Armstrong         92
Donald Reynolds         97
Douglas Tucker          87
Frank Brown             85
George Lewis           104
Jerry Green             85
Joe Price               87
Jonathan Hawkins        81
Joshua Bennett          85
Joshua Little           83
Joshua Ryan             83
Keith Griffin           94
Nicholas Cunningham     88
Patrick Graham          92
Paul Holmes             61
Roger Alexander         82
Roy Rice                75
Samuel Fowler           90
Shawn Cook              88
Shawn Torres            77
Shawn Wallace           95
Stephen Payne           97
Todd Roberts           113
Name: count, dtype: int64


In [18]:
pivot_ordenes = pd.pivot_table(
    df_base,
    index="Sales Team",             
    columns="CustomerDaysInterval", 
    values="OrderNumber",           
    aggfunc="count",                
)

print(pivot_ordenes)


CustomerDaysInterval  0 to 15 days  15 to 30 days  30 to 45 days  \
Sales Team                                                         
Adam Hernandez                  81            179             42   
Anthony Berry                   76            181             41   
Anthony Torres                  81            162             45   
Carl Nguyen                     96            178             40   
Carlos Miller                   65            148             34   
Chris Armstrong                 92            156             44   
Donald Reynolds                 97            163             36   
Douglas Tucker                  87            145             45   
Frank Brown                     85            150             40   
George Lewis                   104            169             42   
Jerry Green                     85            165             46   
Joe Price                       87            138             41   
Jonathan Hawkins                81            14

C:\Users\Legion\AppData\Local\Temp\ipykernel_14468\56129326.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_ordenes = pd.pivot_table(


## 6. Agrega la siguiente columna calculada a la base:
KPI Cálculo


In [20]:
df_base["GrossMargin"] = (
    df_base["Order Quantity"] * (df_base["Unit Price"] * (1 - df_base["Discount Applied"]) - df_base["Unit Cost"])
)
print(df_base["GrossMargin"])

0       4073.4325
1        886.4100
2        905.5050
3       5486.7640
4       1312.1280
          ...    
7986      94.9725
7987    6725.4600
7988    1338.9950
7989    1286.4000
7990    3371.7750
Name: GrossMargin, Length: 7991, dtype: float64


## 7. En esta parte debes aplicar un porcentaje a la columna GrossMargin, 
pero ese porcentaje dependerá del intervalo en que se encuentre GrossMargin


In [22]:
# Clasificar el margen bruto de acuerdo a los intervalos del anexo
gm_intervalos = [0, 100, 1000, 10000, 100000]
gm_etiuquetas = [0.05, 0.10, 0.15, 0.20]  # porcentajes en decimal para poder hacer calculos

df_base["CommissionsPercentage"] = pd.cut(
    df_base["GrossMargin"],
    bins=gm_intervalos,
    labels=gm_etiuquetas,
    right=False,         # incluye el límite inferior, excluye el superior
    include_lowest=True
).astype(float)

# Calcular la comisión final
df_base["CommissionAmount"] = df_base["GrossMargin"] * df_base["CommissionsPercentage"]

# Verificar primeras filas
print(df_base[["GrossMargin", "CommissionsPercentage", "CommissionAmount"]].head())


   GrossMargin  CommissionsPercentage  CommissionAmount
0    4073.4325                   0.15        611.014875
1     886.4100                   0.10         88.641000
2     905.5050                   0.10         90.550500
3    5486.7640                   0.15        823.014600
4    1312.1280                   0.15        196.819200


## 8. Calcula el monto de la comisión, utilizando la Tabla del Anexo.

In [24]:
df_base["CommissionAmount"] = df_base["GrossMargin"] * df_base["CommissionsPercentage"]

print(df_base[["GrossMargin", "CommissionsPercentage", "CommissionAmount"]].head())


   GrossMargin  CommissionsPercentage  CommissionAmount
0    4073.4325                   0.15        611.014875
1     886.4100                   0.10         88.641000
2     905.5050                   0.10         90.550500
3    5486.7640                   0.15        823.014600
4    1312.1280                   0.15        196.819200


## 9. Calcula la comisión sobre el margen bruto.


In [26]:
df_base["NetMargin"] = df_base["GrossMargin"] - df_base["CommissionAmount"]
print(df_base[["GrossMargin", "CommissionsPercentage", "CommissionAmount", "NetMargin"]].head())

   GrossMargin  CommissionsPercentage  CommissionAmount    NetMargin
0    4073.4325                   0.15        611.014875  3462.417625
1     886.4100                   0.10         88.641000   797.769000
2     905.5050                   0.10         90.550500   814.954500
3    5486.7640                   0.15        823.014600  4663.749400
4    1312.1280                   0.15        196.819200  1115.308800


In [27]:
# Crear tabla pivote
pivot_margenes = pd.pivot_table(
    df_base,
    index="Sales Team",  
    values=["GrossMargin", "NetMargin", "CommissionAmount"],  
    aggfunc="sum"  
)

# Redondear a 2 decimales
pivot_margenes = pivot_margenes.round(2)

print(pivot_margenes)


                     CommissionAmount  GrossMargin  NetMargin
Sales Team                                                   
Adam Hernandez              126637.21    779095.20  677987.33
Anthony Berry               124061.18    767943.95  654798.34
Anthony Torres              117962.21    743901.44  636758.05
Carl Nguyen                 123843.21    774781.47  661874.73
Carlos Miller               110882.09    668752.93  580326.02
Chris Armstrong             122565.71    781608.70  660857.42
Donald Reynolds             143768.34    885386.81  754038.26
Douglas Tucker              120806.40    745528.56  633786.05
Frank Brown                 104839.47    670881.25  570076.79
George Lewis                136852.26    839150.75  710824.78
Jerry Green                 126660.44    788587.76  670379.03
Joe Price                   128822.93    787707.44  662662.11
Jonathan Hawkins            126216.94    781391.59  664629.88
Joshua Bennett              108049.11    661726.54  569242.06
Joshua L